In [7]:
import pandas as pd

# Show all columns and rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

google_sheet_column_pasted = """
12.15
-0.0203
16.77
-16.77
16.42
12
10
10.15
50.82
5.46
5.82
5.97
-12.03
-72.42
95.38
45.01
9
14.46
5.24
36.88
5.31
4.64
5.29
5.61
6.28
9.1
6.35
14.83
6.09
6.87
30.92
2.82685
4.74308
25.97402
0.27818
0.29926
31.25
18.44773
26.96628
19.87141
-27.30142
-19
17.92021
45.74563
0.56686
-46.20563
18
6.09013
4.29846
4.08496
0.48524
48.46306
4.04858
8.83743
7.83171
0.07831
8.726
0.08686
0.16163
15.97444
3.07534
0.22096
1.02002
9.42655
13.58695
0.13269
0.08757
8.85044
0.09501
9.50213
9.80392
0.10304
-138.59786
"""

in_order_solana_transacted = pd.DataFrame([float(line) for line in google_sheet_column_pasted.splitlines() if line.strip()])
in_order_solana_transacted.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
0,12.15,-0.0203,16.77,-16.77,16.42,12.0,10.0,10.15,50.82,5.46,5.82,5.97,-12.03,-72.42,95.38,45.01,9.0,14.46,5.24,36.88,5.31,4.64,5.29,5.61,6.28,9.1,6.35,14.83,6.09,6.87,30.92,2.82685,4.74308,25.97402,0.27818,0.29926,31.25,18.44773,26.96628,19.87141,-27.30142,-19.0,17.92021,45.74563,0.56686,-46.20563,18.0,6.09013,4.29846,4.08496,0.48524,48.46306,4.04858,8.83743,7.83171,0.07831,8.726,0.08686,0.16163,15.97444,3.07534,0.22096,1.02002,9.42655,13.58695,0.13269,0.08757,8.85044,0.09501,9.50213,9.80392,0.10304,-138.59786


In [28]:
from itertools import zip_longest

transactions = in_order_solana_transacted[0].tolist()
running_fifo_used = []
next_to_use_index = 0

# Note: Must start with a positive count, and doesn't support overall loss
for changed in transactions:
    print("changed ", changed)
    # Every number should be tracked in the new column.
    if changed >= 0:
        # We just want to start an entry for this number; start at zero to "use up" later.
        running_fifo_used.append(0.0)
    else:
        running_fifo_used.append(float('nan'))
        # Find out how much of the earliest-purchased asset count has already been used as cost basis
        remainder = transactions[next_to_use_index] - running_fifo_used[next_to_use_index] 
        change_to_track = abs(changed)
        # If the change is less than the amount, we don't need to worry about shifting to the next element of the transaction array
        if change_to_track <= remainder:
            running_fifo_used[next_to_use_index] = running_fifo_used[next_to_use_index] + change_to_track
        # If it's more, we need to make sure, so long as there's more change to track, we cycle through the earliest element of the transaction array
        elif change_to_track > remainder:
            to_change = change_to_track - remainder
            print('to change ', to_change)
            while to_change > 0:
                print('enter loop to change', to_change)
                # Remainder already factored in to get here, so just finish up that element of the fifo tracking column
                running_fifo_used[next_to_use_index] = transactions[next_to_use_index]
                next_to_use_index += 1
                # If we encounter a negative, assume it's a previously handled changed value: move on to the next positive
                print('next index ', next_to_use_index)
                while transactions[next_to_use_index] < 0:
                    next_to_use_index += 1
                    print('next index ', next_to_use_index)
                # Check if we are continuing to blast through more purchases...
                if to_change <= transactions[next_to_use_index]:
                    running_fifo_used[next_to_use_index] = running_fifo_used[next_to_use_index] + to_change
                    to_change = 0 # Break out of loop
                    print("break")
                else:
                    to_change = to_change - transactions[next_to_use_index]
df = pd.DataFrame(zip_longest(transactions, running_fifo_used, fillvalue=0), columns=["Transaction History", "Asset used in FIFO cost basis"])
df

changed  12.15
changed  -0.0203
changed  16.77
changed  -16.77
to change  4.6403
enter loop to change 4.6403
next index  1
next index  2
break
changed  16.42
changed  12.0
changed  10.0
changed  10.15
changed  50.82
changed  5.46
changed  5.82
changed  5.97
changed  -12.03
changed  -72.42
to change  72.3203
enter loop to change 72.3203
next index  3
next index  4
enter loop to change 55.9003
next index  5
enter loop to change 43.9003
next index  6
enter loop to change 33.9003
next index  7
enter loop to change 23.750300000000003
next index  8
break
changed  95.38
changed  45.01
changed  9.0
changed  14.46
changed  5.24
changed  36.88
changed  5.31
changed  4.64
changed  5.29
changed  5.61
changed  6.28
changed  9.1
changed  6.35
changed  14.83
changed  6.09
changed  6.87
changed  30.92
changed  2.82685
changed  4.74308
changed  25.97402
changed  0.27818
changed  0.29926
changed  31.25
changed  18.44773
changed  26.96628
changed  19.87141
changed  -27.30142
to change  0.2317200000000028

,Transaction History,Asset used in FIFO cost basis
0,12.15000,12.15000
1,-0.02030,NaN
2,16.77000,16.77000
3,-16.77000,NaN
4,16.42000,16.42000
5,12.00000,12.00000
6,10.00000,10.00000
7,10.15000,10.15000
8,50.82000,50.82000
9,5.46000,5.46000
